# Metryki klasyfikacji

Aby poznać popularne metody oceny klasyfikatorów wygenerujemy sobie przykładowe sekwencje klas przykładów. Wykorzystamy w tym celu bibliotekę numpy oraz generator liczb losowych z rozkładu normalnego.

In [2]:
import numpy as np
from numpy.random import RandomState

In [3]:
random = RandomState(30)

random_1 = random.normal(loc = 0.0, size = 100)
random_2 = random.logistic(loc = 1, size = 100)
# loc = srodek rozkładu
# size = wielkosc proby

In [4]:
print(random_1[:10])
print(random_2[:10])

[-1.26405266  1.52790535 -0.97071094  0.47055962 -0.10069672  0.30379318
 -1.72596243  1.58509537  0.13429659 -1.10685547]
[-0.34625296  2.7465497   0.46063247  3.2000744   0.12353606  1.56724961
  2.32137633 -0.21323077  1.9572528   0.03456367]


In [6]:
y_test = [1 if i >= 0 else 0 for i in random_1]
y_pred = [1 if i >= 0 else 0 for i in random_2]
# list comprehension

In [9]:
print(y_test[:10])
print(y_pred[:10])

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]


## Macierz błędów

Pierwszym krokiem będzie stworzenie macierzy błędów, czyli wyznaczenia true postives, true negatives, false positives i false negatives.

In [10]:
def confusion_matrix(truth, prediction):
    tp, tn, fp, fn = 0, 0, 0, 0
    for label_t, label_p in zip(truth,prediction):
        if label_t == label_p:
            if label_p ==1:
                tp += 1
            else:
                tn += 1
        else:
            if label_p ==1:
                fp += 1
            else:
                fn += 1
    return tp, tn, fp, fn

In [11]:
tp, tn, fp, fn = confusion_matrix(y_test, y_pred)

In [13]:
print(f'TP:{tp}, TN:{tn}')
print(f'Ile obserwacji zaklasyfikowaliśmy poprawnie: {tp+tn}')

TP:30, TN:14
Ile obserwacji zaklasyfikowaliśmy poprawnie: 44


In [14]:
print(f'FP:{fp}, FN:{fn}')
print(f'Ile obserwacji zaklasyfikowaliśmy niepoprawnie: {fp+fn}')

FP:43, FN:13
Ile obserwacji zaklasyfikowaliśmy niepoprawnie: 56


## Accuracy

Pierwszą miarą jest miara dokładności 

accuracy = (tp+tn) / (tp+tn+fp+fn)

In [32]:
def accuracy(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return (tp+tn) / (tp+tn+fp+fn)

Wynik dla analizowanych predykcji

In [23]:
accuracy_1 = accuracy(y_test, y_pred)
print(f'Dokładność: {accuracy_1}')

Dokładność: 0.44


#### Problem
Accuracy zachowuje się źle przy niezbalansowanych zbiorach. Wygenerujemy zbiór, w którym większość przykładów będzie negatywna.

In [28]:
random_3 = random.normal(loc = -1.5, size = 100)
y_pred_2 = [1 if i >= 0 else 0 for i in random_3]

In [29]:
sum(y_pred_2)

6

In [35]:
accuracy_2 = accuracy(y_test, y_pred_2)
print(accuracy_2)

0.55


Zakładając z góry, że wszystkie przykłady są negatywne uzyskujemy bardzo wysoką dokładność. Takie zachowanie jest w wielu przypadkach niepożądane - przypuśćmy, że próbujemy stworzyć klasyfikator do komórek rakowych, gdzie większość przypadków jest negatywna - dla takiego podejścia, zwrócenie informacji że wszystkie przypadki są negatywne da bardzo wysoką dokładność.

In [36]:
accuracy_3 = accuracy(100*[0], y_pred_2)
print(accuracy_3)

0.94


## Recall

Ta miara z kolei mówi o tym, jaką część dodatnich wyników wykrył klasyfikator.

recall = tp / (tp + fn)

In [37]:
def recall(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return tp / (tp + fn)

In [42]:
recall_1 = recall(y_test, y_pred)
recall_1

0.6976744186046512

In [41]:
recall_2 = recall(y_test, y_pred_2)
recall_2

0.046511627906976744

In [44]:
# Recall_2 wykrywa, że jest mały procent 1 (prawdziwych)

## Precision

Jest to miara, która skupia się tylko na przykładach pozytywnych - mówi jaka część wyników wskazanych przez klasyfikator jako dodatnie jest rzeczywiście dodatnia.

precision = tp / (tp+fp)

In [52]:
def precision(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    return tp / (tp+fp)

In [55]:
precision_1 = precision(y_test, y_pred)
precision_1

0.410958904109589

In [56]:
precision_2 = precision(y_test, y_pred_2)
precision_2

0.3333333333333333

# F1 Score

Jest to średnia harmoniczna precyzji i czułości. Ogólnie - im wyższy F1-score tym lepszy jest klasyfikator.

f_score = (2 * precision * recall)/(prec+rec)

In [57]:
def f1_score(truth, prediction):
    prec = precision(truth, prediction)
    rec = recall(truth, prediction)
    return (2 * prec * rec)/(prec+rec)

In [58]:
f1_score_1 = f1_score(y_test, y_pred)
f1_score_1

0.5172413793103448

In [59]:
f1_score_2 = f1_score(y_test, y_pred_2)
f1_score_2

0.08163265306122448

**Zadanie:** Sprawdź funckje f1_score, precision_score, recall_score z modułu scikit learn

In [66]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

In [62]:
recall_score(y_test, y_pred)

0.6976744186046512

In [63]:
precision_score(y_test, y_pred)

0.410958904109589

In [64]:
f1_score(y_test, y_pred)

0.5172413793103448

In [67]:
confusion_matrix(y_test, y_pred)

array([[14, 43],
       [13, 30]], dtype=int64)

In [69]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.25      0.33        57
           1       0.41      0.70      0.52        43

    accuracy                           0.44       100
   macro avg       0.46      0.47      0.43       100
weighted avg       0.47      0.44      0.41       100

